# Лаб 1. Корреляционная система

При создании задания использовались Visual Studio Code и Python 3.8.

## Теория

Данные для рекомендательных систем представляют из себя матрицу $R$ юзер-айтемов, в строках которой находятся пользователи, а в столбцах объекты. В ячейках матрицы находятся оценки пользователей (или какая-то другая метрика $-$ просмотры, покупки и т.д.):

$$
R = \begin{pmatrix}
   r_{1 1} & r_{1 2} & ...     &    \\
   r_{2 1} & r_{2 1} & ...     &     \\
   ...     & ...     & r_{u i} & ... \\
           &         & ...     &     \\
\end{pmatrix}
$$

Матрица $R$ сильно разрежена, то есть большая часть ячеек пуста, поскольку каждый пользователь взаимодействовал с очень малым числом объектов. Задача $-$ предсказать значения в пустых ячейках, то есть получить новую заполненную матрицу $\hat R$, максимально похожую на $R$.

### Обозначения

- $U - $ множество пользователей;
- $I - $ множество айтемов (объектов);
- $I(u)$, где $u \in U -$ множество айтемов, которое оценил пользователь $u$. То есть такие айтемы, для которых в строке $u$ матрицы $R$ не пусто. Аналогично $I(u, v) -$ множество объектов, которые оценили и $u$ и $v$;
- $U(u)$, где $u \in U -$ Множество пользователей, оценивали то же что и $u$, то есть $I(u) \cap I(v) \ne \empty$, где $v \in U(u)$;
- $r_u$, где $u \in U -$ строка матрицы $R$, соответствующая пользователю $u$;
- $r_i$, где $i \in I -$ столбец матрицы $R$, соответствующий айтему $i$;
- $\bar r_u$, где $u \in U -$ среднее значение по всем заполненным оценкам пользователя $u$.


### Корреляционная модель

Один из самых простых способов $-$ использовать взвешенное среднее. То есть, чтобы определить оценку пользователя $u$ айтема $i$, надо усреднить оценки всех пользователей, посмотревших этот фильм (user-based подход). При этом будем учитывать оценку пользователя с большим коэффициентом если пользователь "похож" на нашего:

$$
\hat{r}_{ui} = \frac{
        \sum_{v \in U(u)} S(u, v)\cdot r_v
    } {
        \sum_{v \in U(u)} S(u, v)
    }
$$

Здесь $S$ это функция близости, которая тем больше, чем более "похожи" пользователи друг на друга.

Поскольку оценки разных пользователей могут отличаться $-$ кто-то ставит всем фильмам 9 и 10, а кто-то 0 и 1, можно попытаться устранить проблему, предсказывая не само значение $r_u$, а отклонение от среднего значения $(r_u - \bar{r}_u)$:

$$
\hat{r}_{ui} = \bar{r}_{u} + \frac{
        \sum_{v \in U(u)} S(u, v) \cdot (r_v - \bar r_v)
    } {
        \sum_{v \in U(u)} S(u, v)
    }
$$

Аналогично определяется item-based модель, только усреднение оценок происходит не по пользователям, оценившим этот объект, а по объектам, которые оценил данный пользователь. Теперь $S$ будет функцией близости двух айтемов:

$$
\hat{r}_{ui} = \bar{r}_{i} + \frac{
        \sum_{j \in I(i)} S(i, j) \cdot (r_i - \bar r_i)
    } {
        \sum_{j \in I(i)} S(i, j)
    }
$$


### Функция сходства

Будем рассматривать функцию сходства двух юзером для user-based модели, функции сходства для айтемов определяется аналогично.

#### Косинусная мера сходства

Считаем косинус угла между пользователями в пространстве определяющих их векторов. То есть берём скалярное произведение и делим на длины векторов. Единственное, на что следует обратит внимание, что берутся не все оценки пользователей, а только для тех айтемов, которые они оба оценили:

$$
S(u, v) = \frac{
        \sum_{i \in I(u, v)} r_{ui}r_{vi}
    }{
        \sqrt{\sum_{i \in I(u)} r_{ui}^2}
        \sqrt{\sum_{i \in I(v)} r_{vi}^2}
    }
$$

#### Корреляция Пирсона

"Коэффициент корреляции Пирсона - это ковариация двух переменных, делённая на произведение их стандартных отклонений."

По смыслу примерно то же что предыдущая, но теперь мы опять учитываем не сами значения, а отклонения от среднего:

$$
S(u, v) = \frac{
        \sum_{i \in I(u, v)} (r_{ui} - \bar r_u) (r_{vi} - \bar r_v)
    }{
        \sqrt{\sum_{i \in I(u, v)} (r_{ui} - \bar r_u)^2}
        \sqrt{\sum_{i \in I(u, v)} (r_{vi} - \bar r_v)^2}
    }
$$

### Функция сходства для бинарных данных

А некоторых задачах взаимодействие пользователя с предметом либо есть, либо нет, например, факт покупки чего-то в интернет магазине. Для таких задач существуют отдельные методы.

![Диаграмма пересечения классов](./images/IU.png)

Пусть $|I(u)| = p$, а $|I(v)| = q$, то есть количество объектов, оцененных $u$ равно $p$, а количество объектов, оцененных $v$ равно $q$. А размер пересечения $|I(u) \cap I(v)| = |I(u, v)| = m$

#### Мера близости Жаккара

Очевидно:

$$
S(u, v) = \frac{
        |I(u) \cap I(v)|
    }{
        |I(u) \cup I(v)|
    }
$$

#### Точный тест Фишера

Если предположить что у пользователей $u$ и $v$ оценки генерируются независимо друг от друга, то можно посчитать вероятность того, что размер пересечения будет именно таким, каким он получился $P \{ |I(u) \cap I(v)| = m \}$. Чем меньше эта вероятность, тем больше схожесть пользователей между собой:

$$
S(u, v)
    = -\log P \{ |I(u) \cap I(v)| = m \}
    = -\log \frac{C_{q}^m C_{|I| - q}^{p-m}}{C_{|I|}^{p}}
$$


## Задание

Дан датасет с оценками пользователями фильмов. Реализуйте алгоритм в соответствии с вашим вариантом и порекомендуйте себе фильмы. Сделайте выводы.

В датасете фильмы оценены по пятибальной шкале. Если в вашем варианте используется функция сходства для бинарных данных, используйте факт просмотра фильма (наличие оценки вообще).

### Варианты

1. User-based подход, Косинусная мера сходства.
2. User-based подход, Корреляция Пирсона.
3. User-based подход, Мера близости Жаккара.
4. Item-based подход, Косинусная мера сходства.
5. Item-based подход, Корреляция Пирсона
6. Item-based подход, Мера близости Жаккара.


## Код

Устанавливаем библиотеки

In [1]:
%pip install pandas tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Импорты

In [2]:
# Пандас нам нужен для загрузки csv и работы с матрицей
import pandas as pd
# Это библиотека для визуализации прогресса в питоновском ноутбуке
from tqdm.notebook import tqdm

import math
import time
import numpy as np

Загружаем датасет

In [3]:
# Сам датасет
ratings = pd.read_csv('./ml-latest-small/ratings.csv', delimiter=',')
# Оставляем в таблице только нужные столбцы
ratings = ratings.loc[:, ["userId", "movieId", "rating"]]

# Строка в таблице в датасета это id пользователя, id фильма, рейтинг
# На id пользователей нам плевать, а фильмы хочется смотреть по названиям,
# поэтому загружаем табличку сопоставления названий фильмов их id
movies = pd.read_csv('./ml-latest-small/movies.csv', delimiter=',')

In [4]:
# Делаем таблицу для преобразования имени в id
title_to_id = movies.loc[:, ["title", "movieId"]]
title_to_id.set_index("title", inplace=True)

# Делаем таблицу для преобразования id в имя
id_to_title = movies.loc[:, ["movieId", "title"]]
id_to_title.set_index("movieId", inplace=True)

# Проверяем что всё работает
print(title_to_id.loc[["Iron Man (2008)", "Doctor Who: A Christmas Carol (2010)"], :])
print(id_to_title.loc[[59315, 147376], :])


                                      movieId
title                                        
Iron Man (2008)                         59315
Doctor Who: A Christmas Carol (2010)   147376
                                        title
movieId                                      
59315                         Iron Man (2008)
147376   Doctor Who: A Christmas Carol (2010)


In [5]:
# Мы хотим получить для себя какую-то рекомендацию,
# для этого оценим несколько фильмов по пятибальной шкале
# (аккуратно копипастим имена из датасета)
my_ratings = {
    "Shining, The (1980)": 4.0,
    "Harley Davidson and the Marlboro Man (1991)": 4.0,
    "Fight Club (1999)": 4.0,
    "Cube (1997)": 4.0,
    "Titanic (1997)": 5.0,
    "Troy (2004)": 5.0,
    "Aviator, The (2004)": 5.0,
    "Requiem for a Dream (2000)": 5.0,
    "Green Mile, The (1999)": 5.0,
    "Star Wars: Episode VII - The Force Awakens (2015)": 3.0,
    "Star Wars: The Last Jedi (2017)": 2.0,
    "Last Airbender, The (2010)": 2.0,
    "Mission: Impossible II (2000)": 2.0,

}

# Даём нашему юзеру id, которого нет в датасете
my_user_id = 666

# Докидываем свои оценки в датасет
for m, r in my_ratings.items():
    mid = title_to_id.loc[m]["movieId"]
    row = pd.DataFrame([[my_user_id, mid, r]], columns=["userId", "movieId", "rating"])
    ratings = pd.concat([ratings, row])

In [6]:
# Тут реализуем функцию предсказания.
# Или класс. Главное - предсказать.

def cosine_similarity(u, v):
    dot_product = np.dot(u, v)
    norm_u = np.linalg.norm(u) #np.sqrt(u.dot(u))
    norm_v = np.linalg.norm(v) #np.sqrt(v.dot(v))
    return dot_product / (norm_u * norm_v)

def calculate_cosine_similarity(my_id):
    cosine_similarity_array = {}
    all_users_ids = []
    for index, row in ratings.iterrows():
        if (row['userId'] not in all_users_ids):
            all_users_ids.append(row['userId'])
    all_users_ids.remove(my_id)
    for el in all_users_ids:
        vec_u = np.array([])
        vec_v = np.array([])
        for index, row in ratings.loc[(ratings['userId'] == my_id)].iterrows():
            if (not ratings.loc[(ratings['userId'] == el)&(ratings['movieId'] == row['movieId'])].empty):
                vec_u = np.append(vec_u, ratings.loc[(ratings['userId'] == my_id)&(ratings['movieId'] == row['movieId'])].at[0, 'rating'])
                vec_v = np.append(vec_v, ratings.loc[(ratings['userId'] == el)&(ratings['movieId'] == row['movieId'])].reset_index(drop=True).at[0, 'rating'])
                similarity = cosine_similarity(vec_u, vec_v)
                if (len(vec_u) > 1):
                    cosine_similarity_array[el] = similarity
                else:
                    cosine_similarity_array[el] = 0.0
                #print("cosine_similarity: ", cosine_similarity_array[el])
        if (el not in cosine_similarity_array):
            cosine_similarity_array[el] = 0.0
            #print("cosine_similarity: ", cosine_similarity_array[el])
            
    return cosine_similarity_array
    
def predict(user_id, movie_id):
    pred = 0      
    if (ratings.loc[ratings['userId'] == user_id]["movieId"].isin([movie_id]).any()):
        pred = ratings.loc[(ratings['userId'] == user_id)&(ratings['movieId'] == movie_id)]['rating'].reset_index(drop=True)[0]
    else:
        numerator = 0
        denominator = 0
        for index, row in ratings.loc[(ratings['movieId'] == movie_id)].iterrows():
            numerator += cosine_similarity_array[row['userId']] * (row['rating'] - ratings.loc[ratings['userId'] == row['userId']]["rating"].mean())
            denominator += cosine_similarity_array[row['userId']]
            #print(numerator, denominator)
        if (denominator != 0):
            pred = ratings.loc[ratings['userId'] == my_user_id]["rating"].mean() + numerator / denominator
            #pred = ratings.loc[(ratings['movieId'] == movie_id)]["rating"].mean() + numerator / denominator
        else:
            pred = 0
    return pred

cosine_similarity_array = calculate_cosine_similarity(my_user_id)

Даём предсказание для каждого фильма

In [7]:
#result = {}
#for m in tqdm(np.unique(ratings["movieId"])):
#    result[m] = predict(my_user_id, m)
#result = {k: v for k, v in result.items()}

result = {}
for m in tqdm(np.unique(ratings["movieId"])):
    # убираем фильмы которые уже оценены:
    if (m in ratings.loc[(ratings['userId'] == my_user_id)]["movieId"].tolist()):
        continue
    # отсекаем фильмы у которых мало оценок:
    if (ratings.loc[(ratings['movieId'] == m)]['userId'].size > 20):
        result[m] = predict(my_user_id, m)
    else:
        result[m] = 0 #predict(my_user_id, m) - 5 / ratings.loc[(ratings['movieId'] == m)]['userId'].size
result = {k: v for k, v in result.items()}

  0%|          | 0/9724 [00:00<?, ?it/s]

Выводим результат

In [8]:
# Преобразуем id фильмов в нормальные названия
human_readable_result = {}
for m, v in result.items():
    title = id_to_title.loc[m]["title"]
    human_readable_result[title] = v

# Сортируем массив с результатами по убыванию
sorted_result = {k: v for k, v in sorted(human_readable_result.items(), key=lambda item: item[1], reverse=True) if v != 0.0}

# Выводим первые 100 рекомендаций
for m, v in list(sorted_result.items())[:100]:
    print(m, ": ", v) #print(m, ": ", v * 5 / sorted_result[list(sorted_result.keys())[0]])


Wallace & Gromit: The Best of Aardman Animation (1996) :  4.887974029356768
Lawrence of Arabia (1962) :  4.658531721491893
Roman Holiday (1953) :  4.623106571620098
Shawshank Redemption, The (1994) :  4.617206593301284
Duck Soup (1933) :  4.61449584424617
Patton (1970) :  4.599235189019494
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950) :  4.596427937675051
Good, the Bad and the Ugly, The (Buono, il brutto, il cattivo, Il) (1966) :  4.580203879466552
Grand Day Out with Wallace and Gromit, A (1989) :  4.578963940457579
Producers, The (1968) :  4.576366182007521
Pulp Fiction (1994) :  4.570350096196454
Seven Samurai (Shichinin no samurai) (1954) :  4.560410700070496
My Fair Lady (1964) :  4.550591378175257
Heavenly Creatures (1994) :  4.544145606768649
Godfather, The (1972) :  4.536972302787729
Monty Python and the Holy Grail (1975) :  4.525845762296791
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964) :  4.525831190094447
Boondock Saints, The (2000) :  4.52068932

## Выводы

Объясните полученный результат:

Результат не очень высокого качества, вперед пролезают фильмы с малым количеством оценок, как минимум их стоит отсеивать.
Для достижения лучшего результата стоит учитывать другие факторы, например любимые жанры, отсеивать пользователей с которыми
имеется мало совпадений в просмотренных фильмах при расчете и тд.